Functions for detecting change points in the cross section profiles.
Two approaches: piecewise linear regression, detect peaks in smoothed curvature. 
In this case, curvature is the second derivative of the elevation profile, and not curvature from the elevation map directly. 

Then need to figure out the elevation of the valley bottom from these results.


Useful resources:
- https://gurobi-optimization-gurobi-optimods.readthedocs-hosted.com/en/latest/mods/lad-regression.html
- https://pro.arcgis.com/en/pro-app/latest/tool-reference/space-time-pattern-mining/how-change-point-detection-works.htm#:~:text=Slope%20(Linear%20trend)%E2%80%94Detects,having%20the%20same%20standard%20deviation.
- https://stackoverflow.com/questions/47519626/using-numpy-scipy-to-identify-slope-changes-in-digital-signals
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks_cwt.html
- https://docs.scipy.org/doc/scipy/reference/signal.html
- https://github.com/deepcharles/ruptures


functions to implement:
- slope threshold
- linear regression (ols, lad) two piece
- piece wise analysis
- curvature analysis peak
- curvature analysis waveforem
- change point detection 

In [1]:
# load example signals (elevation and slope profiles)